# Investigate the TMDb Movie Data

## Data Wrangling

In this section, we will firstly load in the data, check for cleanliness, and then trim and clean our dataset for analysis.



### General Properties

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
plt.style.use('seaborn-white')
%matplotlib inline

NOW, import the data and take a brief look at what the data set is like.  

In [ ]:
# load the data
filepath = 'tmdb-movies.csv'
df = pd.read_csv(filepath)

# data records, columns, data type, and missing values check
df.info()

From the above, we can attain a brief summary of the data structure such as how many records, columns are there, which data each column has and which of them have null values. 

Then let's ouptput a few lines to see how these data actually look like in the data set.

In [ ]:
# output the first five rows to see how the data is presented in the data set.
df.head()

From the above, we can see there are 10866 records. There are three data types in total, e.g. ***int***, ***float*** and ***strings***. Some columns like cast, homepage, tagline, production companies have lots of null values. Certain columns, like ‘cast’, ‘genres’ and production companies, contain multiple values separated by pipe ('|') characters. We need to do some tricks on these columns for answering our questions later on.  

Next, let's take a quick view of descriptive statistics summary on the numeric data type in this data set. *See as below*.

In [ ]:
# make a brief descriptive statistics summary on the data
df.describe()

The summary above tells us some important numeric indicators about the movie, such as,  
- the movie data set records the information from 1960 to 2015.
- the average runtime of a movie is 102 mins but the maximum value is 900 mins which needs to be studied on deeper!
- The mimium values of some columns are 0 such as the `budget`, `revenue`. This is against reality. According to the explaination on [Kaggle](https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv), *'it was necessary to treat values of zero in the budget field as missing.'*, so we may consider this when we are answering buget/revenue related questions.

### Data Cleaning
In this section, we are going to do some wrangling on the original data set such as dropping some redundant information, groupping, computing etc. for making our exploratory data analysis and answering our questions easily.

#### Drop the irrelevant columns

As some columns do not make sense to answer our questions, we need to delete them from our dataset. Let's decide which columns need to be dropped.

In [ ]:
# general information about the data in the table.
df.info()

Columns like `id`, `imdb_id`, `cast`, `homepage`, `tagline`, `keywords`, `overview` are all irrelevant to our analysis in the project, so we drop them immediately.

In [ ]:
# drop the irrelevant columns.
df.drop(['id', 'imdb_id', 'cast', 'homepage', 'tagline', 'keywords', 'overview'], axis=1, inplace=True)

In [ ]:
# check the dataset after dropping.
df.head()

#### Drop the missing values
Now let's look at the missing values in the dataset.

In [ ]:
# check which columns have missing values and how many are there?
df.isnull().sum()

From the result above, we can see that within the columns of `director`, `genres`, `production_companies`, there are a few null values, as they do not take up much proportion, we just choose to delete them.

In [ ]:
df.dropna(inplace=True)

In [ ]:
# re-check whether the missing values are successfully dropped or not.
df.isnull().any()

In [ ]:
# reset the index
df.reset_index(inplace=True, drop=True)

#### Alter the data type
Next let's transform the data types of `budget`, `revenue` from ***int*** to ***float***.

In [ ]:
# change the data type of budget, revenue to float
df['budget'] = df['budget'].astype(float)
df['revenue'] = df['revenue'].astype(float)

Now let's check again the descriptive ststistics summary of the data set after cleaning.

In [ ]:
df.describe()

As we can tell from the summary above, the longest runtime of a movie is 877 mins. To confirm this whether it's wrong or not, I searched it on [google](https://www.google.com/search?sxsrf=ALeKk03lXkiNzFw9YwQZ8k338MqIEdPV6A%3A1601458065265&ei=kU90X9_RD4-_0PEP4cCM-A0&q=taken+dreamworks+runtime&oq=taken+dreamworks+runtime&gs_lcp=CgZwc3ktYWIQAzIFCCEQoAEyBQghEKABOgQIIxAnOgQIABAeUP-TB1i1mgdgspsHaABwAHgAgAHYAogBvAmSAQUyLTEuM5gBAKABAaoBB2d3cy13aXrAAQE&sclient=psy-ab&ved=0ahUKEwif3uiYyJDsAhWPHzQIHWEgA98Q4dUDCA0&uact=5), and found out it is a TV mini series, so let's leave as it is for the moment.

About the situation where budget and revenue equals to 0, as there are as large as (***See as below***), we cannot delete from the data right now as such large volumn of data will affect the liability and precision of other non-budget/revenue indicators' analysis. So we will also keep them temperarily here.

In [ ]:
df[(df.budget==0) | (df.revenue==0)]['budget'].count()

In [ ]:
df[(df.budget==0) & (df.revenue==0)]['budget'].count()

#### Add auxiliary columns
We noticed that in the `production companies` and `genres`, the data is joined by pipe '|'. For answering the questions ahead better, here we need to add some new auxiliary columns into the dataframe. We take the elemnt before the first '|' in each record as the primary value, e.g. `primary_genre` and `primary_company`


In [ ]:
# add the primary_company column
df['company_split'] = df['production_companies'].str.split('|')
df['primary_company'] = df['company_split'].apply(lambda x: x[0])

# add the primary_genre column
df['genre_split'] = df['genres'].str.split('|')
df['primary_genre'] = df['genre_split'].apply(lambda x: x[0])

The data is comparatively clean now and we can begin our exploratory analysis during which we may need to do some more transformation and computing as we are trying to answer our questions.

## Exploratory Data Analysis

### Question 1. How is the number of movie made per year changing with the time?  

First, let's group the data by year and sum up the number of movies made per year.  
We will use the `groupby` method to answer this question. As the dataset after cleaning is all non-null recordes, when we are execute `count` on the group, the result of any column is the same, so here we choose an arbitrary column to calculate.

In [ ]:
# extract and calculate the data
num_movies = df.groupby('release_year')['popularity'].count()
num_movies.head()

In [ ]:
# visualize the data
x_tick = [i for i in range(1960,2016,5)]
y_tick = [i for i in range(0, 700, 50)]
num_movies.plot(figsize=(10,6), xticks=x_tick, yticks=y_tick, linewidth=2.3)
plt.xlabel('Release Year', fontsize=12, labelpad=10)
plt.ylabel('Number of Movies', fontsize=12, labelpad=10)
plt.title('Number of movies produced by year', fontsize=18, fontweight='semibold');

From the line plot above, we can see that the number of movies produced in a year is increasing as years went by, especially after 2000, it was rising very sharply, hitting the peak of 638 in 2014. Up unitl 2015, the number made in one year worldwide is almost 14 times as large as that back in 1960.  

### Question 2.  Who are the TOP 5 companies who made the most movies throughout the years?

Before answering this question, we need to add two new columns--'company_split', 'primary_company'. 'company_split' columns is an auxiliary column to split the data in 'production_companies' by '|'; we take the first element or company as the 'primary_company' who mainly produced the movie.

In [ ]:
# aggregate the number of movies produced in history by primary company name
df_numByCo = df.groupby('primary_company')['release_year'].count().sort_values(ascending=False)
df_numByCo

In [ ]:
# extract the top5 most productive companies from above
top5_numByCo = df_numByCo[:5]
top5_numByCo

In [ ]:
# the name of 'Twentieth Century Fox Film Corporation' is long which makes the present on the axis bad, so I shorten it.
new_tickLabel = ['Twentieth Century Fox' if i =='Twentieth Century Fox Film Corporation' else i for i in top5_numByCo.index]

# define an appropiate range for y axis ticks
y_tick = [i for i in range(0, 550, 50)]

# draw a bar chart for visualizing the question
top5_numByCo.plot(kind='bar', width=0.3, grid=False, figsize=(14,6), color='grey')

# adjust the graph to make it look user-friendly
plt.title('TOP 5 MOST PRODUCTIVE COMPANIES', fontsize=18, fontweight='semibold')
plt.xlabel('Production Company', fontsize=13, labelpad=10, fontweight='semibold')
plt.ylabel('Number of Movies', fontsize=13, labelpad=10, fontweight='semibold')
plt.xticks(np.arange(5), new_tickLabel, rotation=0,  fontsize=12);
plt.yticks(y_tick, fontsize=10);

From the bar chart above, we can see the top 5 companies who made the most movies in history.

Next, we can even take a look at how the number of movies made in each company varied with time.

In [ ]:
# construct the dataframe for visualization.
top5_numByCo_names = list(top5_numByCo.index)
df_top5_numByCo = df[df['primary_company'].isin(top5_numByCo_names)]
Stat_top5_numByCo = df_top5_numByCo.groupby(['primary_company','release_year'])['original_title'].count()
Stat_top5_numByCo.name = 'count'

# reconstructure the DataFrame for visualization purpose
Stat_top5_numByCo1 = Stat_top5_numByCo.unstack(level=0)
Stat_top5_numByCo1.head()

In [ ]:
# visualize the movie number variation by each year among the 5 most prodcutive companies
colour_pa = ['#DF5427','#A6B827','#F69200','#418AB3', '#838383']
for i, j, k in zip(np.arange(511, 516), top5_numByCo_names, colour_pa):
    plt.subplot(i)
    Stat_top5_numByCo1[j].plot(linestyle='--', linewidth=2, marker='o', figsize=(18,30), color=k)
    plt.title("Yearly Production Detail by {}".format(j), fontweight='semibold', fontsize=15)
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number', fontsize=12)
    plt.grid(color='grey', linewidth=0.4, alpha=0.5)
    plt.xticks(np.arange(1960, 2016), rotation=70)
    plt.xlim(1959,2016)
    subplots_adjust(hspace=0.4)

From all the pictures above, we can find that after 1990, the number of movies made each year in all the five companies were jumping onto a higher level.

### Question 3.  What are the 5 movies which made the most revenues in history?

In [ ]:
# create the DataFrame fot visualizing the 5 movies who made the most revenues of all time
top5_rev = df.sort_values('revenue', ascending=False)[['original_title','revenue']][:5]
top5_rev.reset_index(inplace=True, drop=True)

# create the figure and make it more readible
colour = ['#24D09C','#276FC6','#2D9ED9','#30D0D9','#F59100']
new_tickLabel1 = ['Star Wars: \nThe Force Awakens' if i =='Star Wars: The Force Awakens' else i for i in top5_rev.original_title[::-1]]
top5_rev.sort_values('revenue').plot(y='revenue', x='original_title', width=0.5, kind='barh', color = colour, figsize=(10,7), legend=None)
plt.title('TOP 5 MOVIES WITH MOST REVENUES', fontsize=18, fontweight='semibold')
plt.ylabel('MOVIE NAME', fontsize=13, labelpad=10)
plt.xlabel('REVENUE', fontsize=13, labelpad=10)
plt.yticks(np.arange(5), new_tickLabel1, rotation=0, fontsize=12, fontweight='semibold')
plt.xticks(np.arange(0,3.5*10**9,0.4*10**9), fontsize=11)
plt.grid(axis='x', color='k', linewidth=0.3, linestyle=':', alpha=0.9);

From the horizontal bar chart above, we can see that the 5 movies with the most revenue in history are:  
1. Avatar
2. Star Wars: The Force Awakens
3. Titanic
4. The Avengers
5. Jurassic World

### Question 4. What are the most popular movie types  companies would like to make?

In [ ]:
# aggregate the data by movie genre and sum up the numbers in each genre respectively.
df_numByType = df.groupby('primary_genre')['original_title'].count().sort_values(ascending=False)

# draw the graph
df_numByType.plot(kind='bar', width=0.4, color='grey', grid=False, figsize=(15,6))
plt.xlabel('Primary Genre', fontsize=12, fontweight='semibold')
plt.ylabel('Number', fontsize=12, fontweight='semibold', labelpad=10)
plt.title('Total Number of Movies Produced by Genre', fontsize=18, fontweight='bold')
plt.xticks(rotation=30, ha='right')
plt.yticks([])

# get rid of the surrounding axis frames
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

# add the annotation on top of the bar chart
total_type = len(df_numByType.index)
for i, dt in zip(np.arange(total_type), df_numByType):
    plt.text(x=i-0.3, y=dt+30, s=str(dt), fontsize=12)


From the bar chart above, we can see that three genres, namely ***drama***, ***comedy***, ***action***, is overwhelmingly popular, based on the their numbers already produced throughout the years.   

Next we'll see how the production of these 3 genres, varied with the time.

In [ ]:
# extract the 3 most popular genres
top3_genre_names = list(df_numByType.index)[:3]
top3_genre_names

# extract the 5 most popular genres
top5_genre_names = list(df_numByType.index)[:5]
top5_genre_names

In [ ]:
# create the top 3 genres series
years = np.arange(1960, 2016)
drama_srs = df[df['primary_genre']=='Drama'].groupby('release_year')['original_title'].count().values
comedy_srs = df[df['primary_genre']=='Comedy'].groupby('release_year')['original_title'].count().values
action_srs = df[df['primary_genre']=='Action'].groupby('release_year')['original_title'].count().values

labels = ['Drama', 'Comedy', 'Action']

# make the stackplot
plt.figure(figsize=(16,6))
plt.stackplot(years, drama_srs, comedy_srs, action_srs, labels=labels)

plt.xticks(years[::2])

plt.xlim(1960,2015)
plt.legend(loc='upper left');

From the area stacked graph above, we can see that the number of movies in each movie genre was increasing year by year, with ***drama*** covering the largest proportion and ***action*** relatively smaller.

In [ ]:
# get the series for the 5 most prodcutive companies
top5_CoByGenre_list = ["srs_{}".format(i) for i in top5_numByCo_names ]

# use an empty list to store the data from the top 5 companies' production detail by genres
srs_list = []
for i, j in zip(top5_CoByGenre_list, top5_numByCo_names):
    i = df[df['primary_company']==j].groupby('primary_genre')['original_title'].count()
    srs_list.append(i)

# concatenate these data into a new DataFrame
new_df = pd.concat(srs_list, axis=1, sort=False)

# rename the columns of the new DataFrame
new_df.columns = top5_numByCo_names

# transpose the DataFrame for the sake of visualization later on
tmp = new_df.T[top5_genre_names]

# Check the new data
tmp

Now let's explore more a little bit.  
I am wondering among the top 5 most productive companies, how the 5 most preferred movie genre varies each other. Here are the steps.  
1. extract the 5 most preferred movie genre. We can refer to the result of the previous question.
2. extract the 5 most productive companies.
3. Based on each company, create a stacked bar chart consisting the 5 genres from step 1.

In [ ]:
# draw the stacked bar chart
x = np.arange(len(top5_numByCo_names))
plt.figure(figsize=(14,6))
bar_width = 0.4
color_pa = ['#DF5427','#2D9ED9','#A6B827','#F69200','#838383']
plt.bar(x, tmp['Drama'], width = bar_width, label='Drama', color=color_pa[0])

for i, j, c in zip(np.arange(1, len(list(tmp.columns)[1:5])+1), list(tmp.columns)[1:5], color_pa[1:]):    
    plt.bar(x, 
            tmp["{}".format(j)], 
            bottom=tmp[[k for k in list(tmp.columns)[:i]]].sum(axis=1), 
            width = bar_width, 
            label="{}".format(j), color=c)
    
# visualization customization    
plt.xticks(x, new_tickLabel, fontweight='semibold', fontsize=12)
plt.legend(title='GENRES', loc='upper right')
plt.title('the genre variation among the top 5 companies'.title(), fontsize=18, fontweight='semibold')
plt.xlabel('Companies', fontsize=14, labelpad=20)
plt.ylabel('Number', fontsize=14, labelpad=20);